In [102]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
# load the data

df = pd.read_csv('student3-5_recoded.csv')

In [3]:
# look at the data
df.head()

,Unnamed: 0,ClientName,ClientName_1,SchoolName,SchoolID,UniqueResponseID,SurveyType,ResponseLanguage,ResponseStartTime,ResponseDuration(minutes),...,QT5_62,QT5_63,QT5_64,SOCFFEJ,race,gender,AAM,SPED,English_speaking,Grade
0,1,Seattle (WA),Seattle Public Schools,Cascade Parent Partnership,945,60132424,Online,en,2020-10-17T21:49:51+00:00,2.757033,...,NaN,NaN,NaN,NaN,NaN,Female,NaN,non-SPED,English,NaN
1,2,Seattle (WA),Seattle Public Schools,Cascade Parent Partnership,945,60132759,Online,en,2020-10-18T01:00:38+00:00,7.541550,...,NaN,NaN,NaN,NaN,NaN,Female,NaN,NaN,English,High
2,3,Seattle (WA),Seattle Public Schools,Stevens Elementary,272,60200632,Online,en,2020-10-19T15:36:33+00:00,4.488667,...,NaN,NaN,NaN,non_SOCFFEJ,White,Male,NaN,NaN,NaN,NaN
3,4,Seattle (WA),Seattle Public Schools,Stevens Elementary,272,60200645,Online,en,2020-10-19T15:36:47+00:00,4.441950,...,NaN,NaN,NaN,non_SOCFFEJ,White,Female,NaN,NaN,English,NaN
4,5,Seattle (WA),Seattle Public Schools,Stevens Elementary,272,60200951,Online,en,2020-10-19T15:36:59+00:00,5.078833,...,NaN,NaN,NaN,non_SOCFFEJ,White,Male,NaN,non-SPED,English,High


In [4]:
df.columns

Index(['Unnamed: 0', 'ClientName', 'ClientName_1', 'SchoolName', 'SchoolID',
       'UniqueResponseID', 'SurveyType', 'ResponseLanguage',
       'ResponseStartTime', 'ResponseDuration(minutes)', 'QT1_1', 'QT1_2',
       'QT2_3', 'QT2_4', 'QT2_5', 'QT2_6', 'QT2_7', 'QT2_8', 'QT3_9', 'QT3_10',
       'QT3_11', 'QT3_12', 'QT3_13', 'QT3_14', 'QT3_15', 'QT3_16', 'QT3_17',
       'QT3_18', 'QT3_19', 'QT3_20', 'QT3_21', 'QT4_22', 'QT4_23', 'QT4_24',
       'QT4_25', 'QT4_26', 'QT4_27', 'QT4_28', 'QT4_29', 'QT4_30', 'QT4_31',
       'QT4_32', 'QT4_33', 'QT4_34', 'QT4_35', 'QT4_36', 'QT4_37', 'QT4_38',
       'QT4_39', 'QT4_40', 'QT4_41', 'QT4_42', 'QT4_43', 'QT4_44', 'QT4_45',
       'QT4_46', 'QT4_47', 'QT4_48', 'QT4_49', 'QT4_50', 'QT4_51', 'QT4_52',
       'QT4_53', 'QT4_54', 'QT4_55', 'QT4_56', 'QT4_57', 'QT4_58', 'QT5_59',
       'QT5_60', 'QT5_61', 'QT5_62', 'QT5_63', 'QT5_64', 'SOCFFEJ', 'race',
       'gender', 'AAM', 'SPED', 'English_speaking', 'Grade'],
      dtype='object')

In [59]:
# The DV are QT2 ~ QT3, IV are gender, English, SOCFFEJ, SPED, LGBTQ.
# Extract the columns
df_ = df[['QT2_3', 'QT2_4', 'QT2_5', 'QT2_6', 'QT2_7', 'QT3_9', 'QT3_10',
       'QT3_11', 'QT3_12', 'QT3_13', 'QT3_14', 'QT3_15', 'QT3_16', 'QT3_17',
       'QT3_18', 'QT3_19', 'QT3_20', 'gender', 'SPED', 'English_speaking','SOCFFEJ']]

In [61]:
# check missing values
# df_.isnull().sum()

In [66]:
print(df_['gender'].value_counts())
print()
print(df_['SPED'].value_counts())
print(df_['English_speaking'].value_counts())
print()
print(df_['SOCFFEJ'].value_counts())


Female    4136
Male      3912
Name: gender, dtype: int64

non-SPED    3084
SPED         772
Name: SPED, dtype: int64
English        5082
non_English    1294
Name: English_speaking, dtype: int64

non_SOCFFEJ    5045
SOCFFEJ        2911
Name: SOCFFEJ, dtype: int64


In [87]:
# replace categorical names as 1 and 0 for two groups so that they have the same values for the loop. 
df_ = df_.replace({'gender': {'Female':0, 'Male': 1},
            'SPED': {'SPED': 1, 'non-SPED': 0},
            'English_speaking': {'English': 1, 'non_English': 0},
            'SOCFFEJ':{'SOCFFEJ':1, 'non_SOCFFEJ':0}})

In [88]:
# find independent variables X, and dependent variables y. 
X_variables = ['gender', 'SPED', 'English_speaking','SOCFFEJ']
y_variables = ['QT2_3', 'QT2_4', 'QT2_5', 'QT2_6', 'QT2_7', 'QT3_9', 'QT3_10',
       'QT3_11', 'QT3_12', 'QT3_13', 'QT3_14', 'QT3_15', 'QT3_16', 'QT3_17',
       'QT3_18', 'QT3_19', 'QT3_20']

In [106]:
# loop all depedent variables and independent variables for t-tests
for x in X_variables:
    for y in y_variables:
        # make a new dataframe for each combination of IV and DV
        new_df = df_[[x, y]]
        # seperate group1 and group2 for t test
        samp1 = new_df[new_df[x]==1][y]
        samp2 = new_df[new_df[x]==0][y]
#       sample mean and std
        samp1_mean = samp1.mean()
        samp1_std = samp1.std()
        samp2_mean = samp2.mean()
        samp2_std = samp2.std()
    
        # conduct t test for each subdataset
        stats, p_value = ttest_ind(samp1, samp2, equal_var = False, nan_policy='omit')
        print(f't-test for {x} and {y}')
        print(f'group1 mean: {samp1_mean.round(2)}; group1 std: {samp1_std.round(2)} ')
        print(f'group2 mean: {samp2_mean.round(2)}; group2 std: {samp2_std.round(2)} ')
        print(f'statistic: {stats.round(3)}, p_value: {p_value.round(3)}')
        print()

t-test for gender and QT2_3
group1 mean: 0.73; group1 std: 0.63 
group2 mean: 0.76; group2 std: 0.58 
statistic: -1.987, p_value: 0.047

t-test for gender and QT2_4
group1 mean: 0.39; group1 std: 0.6 
group2 mean: 0.43; group2 std: 0.58 
statistic: -3.063, p_value: 0.002

t-test for gender and QT2_5
group1 mean: 0.53; group1 std: 0.65 
group2 mean: 0.57; group2 std: 0.63 
statistic: -2.806, p_value: 0.005

t-test for gender and QT2_6
group1 mean: 0.4; group1 std: 0.62 
group2 mean: 0.35; group2 std: 0.58 
statistic: 3.146, p_value: 0.002

t-test for gender and QT2_7
group1 mean: 0.23; group1 std: 0.54 
group2 mean: 0.19; group2 std: 0.49 
statistic: 2.869, p_value: 0.004

t-test for gender and QT3_9
group1 mean: 0.61; group1 std: 0.67 
group2 mean: 0.54; group2 std: 0.62 
statistic: 4.416, p_value: 0.0

t-test for gender and QT3_10
group1 mean: 0.45; group1 std: 0.62 
group2 mean: 0.45; group2 std: 0.59 
statistic: 0.55, p_value: 0.582

t-test for gender and QT3_11
group1 mean: 0.49; g